# Final Project Exploratory Data Analysis


## Project Title: The Impact of Urban Planning and Design on Obesity Levels
- Andrew Noonan
- AJN104@pitt.edu

### Project Abstract
Research question: What is the impact of Urban Planning/Walkability on Obesity Levels within Allegheny County?
This is interesting to me because it allows me investigate whether a statistically significant contributor to the rise in Obesity within the United States is due to the historical transition away from cities that were easily navigable by foot as well as public transportation like trains, to cities that are increasing only able to be traversed by car.I consider this project useful because we are increasingly facing a public health crisis with the rise in obesity in the United States. If it can be proven that cities that are more friendly to alternative means of transportation decrease obesity rates, perhaps that will sway policy makers to utilize another technique in improving Public Health. Leading from that, I believe policy makers would certainly benefit from my model.

In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
walk_score = pd.read_csv('walkscorect.xls-walk-score-by-ct.csv')
obesity_rates = pd.read_csv('obesity-ac-2006-2010censustracts.csv')
transit = pd.read_csv('prt.csv')

In [3]:
walk_score.isnull().sum()
walk_score.head(25)


,Census Tract,Walk Score
0,10300,82
1,20100,100
2,20300,83
3,30500,77
4,40200,71
5,40400,82
6,40500,78
7,40600,75
8,40900,68
9,50100,60


Dropping Null Values

In [4]:
obesity_rates.isnull().sum()
obesity_rates = obesity_rates.dropna()
obesity_rates.isnull().sum()
obesity_rates = obesity_rates.drop(['stname','MUNICIPALITY','tractfips', 'Municipality'], axis=1) # drop columns that are not needed
obesity_rates.head(25)

,2000 Tract,2010 Tract,City Neighborhood,2006-2010 estimate of obesity
59,151500,562900,Hazelwood,0.399853
364,160300,561600,Arlington,0.334520
365,160400,561600,Arlington Heights,0.473120
366,160600,561700,Saint Clair,0.699500
367,160700,561700,Mount Oliver Neighborhood,0.380567
368,120100,"561900, 980000 & 981800",Lincoln-Lemington-Belmar,0.637099
369,40300,562000 & 982200,North Oakland,0.070353
370,150100,562300,Hazelwood,0.371847
371,150400,562300,Glen Hazel,0.383138
372,180600,562400,Bon Air,0.289505


Choosing to drop null values

In [5]:

transit.isnull().sum()
transit = transit.dropna()
transit.isnull().sum()
# drop columns that are not needed
transit = transit.drop(['Direction','CleverID','Direction', 'Routes_ser', 'Routes_cou', 'Mode', 'Shelter', 'Stop_type', 'On_avwk_FY', 'Off_avwk_F', 'On_avwk__1', 'Off_avwk_1', 'AvgWkdy_De', 'Timepoint'], axis=1)
transit.head(25)

,FID,StopID,Stop_name,Latitude,Longitude,x,y
0,1,E41755,NEGLEY AVE AT EAST BUSWAY STEPS,40.456073,-79.933320,-79.933320,40.456073
1,2,E41760,NEGLEY AVE AT EAST LIBERTY BLVD,40.467388,-79.928150,-79.928150,40.467388
2,3,E41770,NEGLEY AVE AT ELLSWORTH AVE,40.454623,-79.932406,-79.932406,40.454623
3,4,E41771,NEGLEY AVE AT ELLSWORTH AVE,40.454766,-79.932659,-79.932659,40.454766
4,5,E41780,NEGLEY AVE AT ELMER ST,40.453376,-79.931705,-79.931705,40.453376
5,6,E41781,NEGLEY AVE AT ELMER ST FS,40.453283,-79.931823,-79.931823,40.453283
6,7,E41790,NEGLEY AVE AT FRIENDSHIP AVE,40.460837,-79.932492,-79.932492,40.460837
7,8,E41800,NEGLEY AVE AT FRIENDSHIP AVE,40.460442,-79.932349,-79.932349,40.460442
8,9,E41810,NEGLEY AVE AT HAMPTON ST,40.475643,-79.924655,-79.924655,40.475643
9,10,E41820,NEGLEY AVE AT HAMPTON ST,40.475199,-79.924472,-79.924472,40.475199


### Why I chose to drop missing values
I choose to drop missing values, because for most of the missing values in this dataset an average doesn't make sense.

In [7]:
census_tracts = gpd.read_file('PAAC_Routes_1909.shp')

ImportError: The 'read_file' function requires the 'pyogrio' or 'fiona' package, but neither is installed or imports correctly.
Importing fiona resulted in: libicuuc.so.74: cannot open shared object file: No such file or directory
Importing pyogrio resulted in: No module named 'pyogrio'